In [1]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import uuid
import os

parsedCount = 0
pageCount = 0
rcpCount = 916

def build_rcp_initial(book, chap):
    id = f'ck-{uuid.uuid4()}'
    fr = 'http://fruschtique.de/ns/recipe'
    xsi = 'http://www.w3.org/2001/XMLSchema-instance'

    recipe = ET.Element('fr:recipe', attrib={'xmlns:fr' : fr, 'xmlns:xsi': xsi, 'xsi:schemaLocation': fr + ' file:///c:/Users/nlutt/Documents/Websites/tools/recipe.xsd', 'rcpID': id})
    meta = ET.SubElement(recipe, 'fr:meta')
    ET.SubElement(meta, 'fr:book').text = book
    ET.SubElement(meta, 'fr:chapter').text = chap
    ET.SubElement(recipe, 'fr:recipeName').text = chap
    ET.SubElement(recipe, 'fr:recipeKeywords')
    ET.SubElement(recipe, 'fr:recipeIntro')
    recipe_ingredients = ET.SubElement(recipe, 'fr:recipeIngredients')
    igdt_list = ET.SubElement(recipe_ingredients, 'fr:igdtList')
    ET.SubElement(igdt_list, 'fr:igdtListName')
    igdt_list_line = ET.SubElement(igdt_list, 'fr:igdtListLine')
    ET.SubElement(igdt_list_line, 'fr:igdtQuantity')
    ET.SubElement(igdt_list_line, 'fr:igdtName')
    instructions = ET.SubElement(recipe, 'fr:recipeInstructions')
    instruction = ET.SubElement (instructions,'fr:instruction')
    ET.SubElement(instruction,'fr:instrStepName')
    ET.SubElement(instruction,'fr:instrStepText')
    ET.SubElement(recipe, 'fr:recipeSideDish')
    ET.SubElement(recipe, 'fr:recipeOrigin')
    ET.SubElement(recipe, 'fr:recipeSeeAlso')
    ET.SubElement(recipe, 'fr:recipeLicense')

    os.chdir(book)
    xml_rcp = ET.ElementTree(recipe)
    file_name = f'{chap}.xml'
    xml_rcp.write(file_name, xml_declaration=True, encoding='utf-8', method='xml')

def build_rcp(book, chap, name, url, _id):
    response = requests.get(url)
    body = response.text
    fr = 'http://fruschtique.de/ns/recipe'
    xsi = 'http://www.w3.org/2001/XMLSchema-instance'

    recipe = ET.Element('fr:recipe', attrib={'xmlns:fr' : fr, 'xmlns:xsi': xsi, 'xsi:schemaLocation': fr + ' file:///c:/Users/nlutt/Documents/Websites/tools/recipe.xsd', 'rcpID': _id})
    meta = ET.SubElement(recipe, 'fr:meta')
    ET.SubElement(meta, 'fr:book').text = book
    ET.SubElement(meta, 'fr:chapter').text = chap
    ET.SubElement(recipe, 'fr:recipeName').text = name
    ET.SubElement(recipe, 'fr:recipeKeywords')
    ET.SubElement(recipe, 'fr:recipeIntro')
    recipe_ingredients = ET.SubElement(recipe, 'fr:recipeIngredients')
    igdt_list = ET.SubElement(recipe_ingredients, 'fr:igdtList')
    ET.SubElement(igdt_list, 'fr:igdtListName')
    
    root = BeautifulSoup(body, 'html.parser')
    for i in range(len(root.select('.ingredients tbody tr'))):
        igdt_list_line = ET.SubElement(igdt_list, 'fr:igdtListLine')
        x = root.select('.ingredients tbody tr')[i].select('td')[0].get_text(strip=True)
        ET.SubElement(igdt_list_line, 'fr:igdtQuantity').text = x
        y = root.select('.ingredients tbody tr')[i].select('td')[1].get_text(strip=True)
        ET.SubElement(igdt_list_line, 'fr:igdtName').text = y

    instructions = ET.SubElement(recipe, 'fr:recipeInstructions')
    instruction = ET.SubElement (instructions,'fr:instruction')
    ET.SubElement(instruction,'fr:instrStepName')
    ET.SubElement(instruction,'fr:instrStepText')
    ET.SubElement(recipe, 'fr:recipeSideDish')
    ET.SubElement(recipe, 'fr:recipeOrigin')
    ET.SubElement(recipe, 'fr:recipeSeeAlso')
    ET.SubElement(recipe, 'fr:recipeLicense')

    xml_rcp = ET.ElementTree(recipe)
    file_name = name.replace('"', '').replace('/', '') + '.xml'
    xml_rcp.write(file_name, xml_declaration=True, encoding='utf-8', method='xml')

def parse_search_result_page(url, book, chap):
    response = requests.get(url)
    body = response.text
    root = BeautifulSoup(body, 'html.parser')
    count_parsed = 0
    for el in root.select('.recipe-list>div'):
        rcp_name = el.get('data-vars-recipe-title')
        rcp_url = el.find('a')
        if rcp_name is not None:
            print(rcp_name)
            count_parsed += 1
            print(count_parsed)

    global rcpCount
    rcpCount -= count_parsed
    print(rcpCount)

def recipe_list(book, chap_name, pC):
    id = 0
    url = f'https://www.chefkoch.de/rs/s{pC}/{chap_name}/Rezepte.html'
    response = requests.get(url)
    body = response.text
    root = BeautifulSoup(body, 'html.parser')
    list_divs = root.select('.recipe-list>div')
    parsed_count = 0
    for el in list_divs:
        rcp_name = el.get('data-vars-recipe-title')
        if rcp_name is not None:
            print(rcp_name)
            rcp_url = el.find('a').get('href')
            parsed_count += 1
            id = el.get('data-vars-tracking-id')
            build_rcp(book, chap_name, f'{id} - {rcp_name}', rcp_url, id)

    global rcpCount
    rcpCount -= parsed_count

if __name__ == "__main__":
    chap_num = int(input("Enter chapter number: "))
    chap_name = input("Enter chapter name: ")
    path = f'c:/users/nlutt/documents/websites/ckCollections/recipes_xml/{chap_num} {chap_name}/'
    os.makedirs(path, exist_ok=True)
    os.chdir(path)
    book = 'ckCollections'
    chap = f'{chap_num} {chap_name}'
    while pageCount < 18:
        recipe_list(book, chap_name, pageCount)
        pageCount += 1
        print(rcpCount)

ValueError: invalid literal for int() with base 10: ''